In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing


2023-01-08 19:34:31.791075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:34:31.901453: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-08 19:34:31.904950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 19:34:31.904961: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Define sampling layer

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z """

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder and decoder

In [3]:
latent_dim = 3
inputlen = 9024
units = 1180

# Encoder
encoder_inputs = keras.Input(shape=(inputlen,))
x = layers.Dense(units=units, activation='relu')(encoder_inputs)
x = layers.Dense(units=units//2, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=units//2, activation='relu')(latent_inputs)
x = layers.Dense(units=units, activation='relu')(x)
decoder_outputs = layers.Dense(inputlen, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 9024)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 1180)         10649500    ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 590)          696790      ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 3)            1773        ['dense_1[0][0]']                
                                                                                            

2023-01-08 19:34:33.798058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-08 19:34:33.798076: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 19:34:33.798090: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2023-01-08 19:34:33.798248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Define VAE

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #print("reconstruction = " + str(reconstruction))
            #print("z_mean = " + str(z_mean)+"\n")
            #print("z_log_var = " + str(z_log_var)+"\n")
            #print("z = " + str(z)+"\n")
            
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            #print("First kl_loss = "+str(kl_loss)+"\n")
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss,axis=1)) 
            #mprint("Second kl_loss = "+str(kl_loss)+"\n")
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Load dataset

In [5]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MetaGenAutoencoder/Data/KO_metaG.norm.txt",sep="\t")
df = df.iloc[:,2-len(df.columns):]



# Train VAE

In [6]:


tb = keras.callbacks.TensorBoard(log_dir="./vae_tb",update_freq="epoch")
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=500)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(df, epochs=5000, batch_size=128, callbacks=[tb,earlystop] )
vae.save_weights('./vae_checkpoint/my_checkpoint')

Epoch 1/5000
2/2 [==============================] - 1s 51ms/step - loss: 7.7959 - reconstruction_loss: 0.0586 - kl_loss: 19.7005
Epoch 2/5000
2/2 [==============================] - 0s 41ms/step - loss: 3.1561 - reconstruction_loss: 0.0513 - kl_loss: 3.9724
Epoch 3/5000
2/2 [==============================] - 0s 43ms/step - loss: 3.8617 - reconstruction_loss: 0.0316 - kl_loss: 2.8633
Epoch 4/5000
2/2 [==============================] - 0s 41ms/step - loss: 0.2568 - reconstruction_loss: 0.0257 - kl_loss: 0.2348
Epoch 5/5000
2/2 [==============================] - 0s 43ms/step - loss: 0.6476 - reconstruction_loss: 0.0235 - kl_loss: 0.5877
Epoch 6/5000
2/2 [==============================] - 0s 40ms/step - loss: 0.3661 - reconstruction_loss: 0.0186 - kl_loss: 0.2729
Epoch 7/5000
2/2 [==============================] - 0s 42ms/step - loss: 0.0970 - reconstruction_loss: 0.0175 - kl_loss: 0.0822
Epoch 8/5000
2/2 [==============================] - 0s 40ms/step - loss: 0.1796 - reconstruction_loss: 